In [ ]:
import docs
from minsearch import Index

from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
github_data = docs.read_github_data()
pased_data = docs.parse_data(github_data)
doc_chunks =docs.chunk_documents(pased_data)

In [7]:
chunks[65]

{'start': 2000,
 'content': '\n\n- **Export scores** as JSON or Python dictionary.\n- **As a DataFrame**, either as a raw metrics table or by attaching scores to existing data rows.\n- **Generate visual reports** in Jupyter, Colab, or export as HTML\n- **Upload to Evidently Platform** to track evaluations over time\n\nThis exportability makes it easy to integrate Evidently into your existing workflows and pipelines –\xa0even if you are not using the Evidently Platform.\n\nHere is an example visual report showing various data quality metrics and test results. Other evaluations can be presented in the same way, or exported as raw scores:\n\n![](/images/concepts/report_test_preview.gif)\n\n**📌 Links:**\n\n- Quickstart for [LLM evaluation](/quickstart_llm) \n- Quickstart for [ML evaluation](/quickstart_ml)\n\nOr read on through this page for conceptual introduction.\n\n**2. Synthetic data generation [NEW]**\n\n<Check>\n  **TL;DR**: We have a nice config for structured synthetic data genera

In [8]:
index = Index(
    text_fields=['content', 'filename','title','description'],
)
index.fit(chunks)

In [9]:
search_results = index.search("data drift detection")

In [13]:
es_client.info()

ObjectApiResponse({'name': 'fc0e6ce77977', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'GmgLnJSiRpGtgAw6PYkDTA', 'version': {'number': '9.1.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '5e94055934defa56e454868b7783b2a3b683785e', 'build_date': '2025-08-05T01:07:31.959947279Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [14]:
index_settings = {
    "mappings": {
        "properties": {
            "start": {"type": "integer"},
            "content": {"type": "text"},
            "title": {"type": "text"},
            "description": {"type": "text"},
            "filename": {"type": "text"}
        }
    }
}

In [15]:
index_name = "evidently-docs"
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'evidently-docs'})

In [18]:
from tqdm.auto import tqdm

for doc in tqdm(doc_chunks):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/575 [00:00<?, ?it/s]

In [21]:
def elastic_search(query, num_results=15):
    es_query = {
        "size": num_results,
        "query": {
            "multi_match": {
                "query": query,
                "type": "best_fields",
                "fields": ["content", "filename", "title", "description"],
            }
        }
    }

    response = es_client.search(index=index_name, body=es_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [22]:
def search(query):
    results = elastic_search(
        query=query,
        num_results=15
    )

    return results

In [23]:
query = "data drift detection"
search_results = search(query)

In [24]:
search_results

[{'start': 7000,
  'content': 'ch as articles.\n\n<Tip>\n  **Text descriptors drift**. If you work with raw text data, you can also check for distribution drift in text descriptors (such as text length, etc.) To use this method, first compute the selected [text descriptors](/docs/library/descriptors). Then, use numerical / categorical drift detection methods as usual.\n</Tip>\n\n\n## Resources\n\nTo build up a better intuition for which tests are better in different kinds of use cases, you can read our in-depth blogs with experimental code:\n\n* [Which test is the best? We compared 5 methods to detect data drift on large datasets](https://evidentlyai.com/blog/data-drift-detection-large-datasets).\n\n* [Shift happens: how to detect drift in ML embeddings](https://www.evidentlyai.com/blog/embedding-drift-detection).\n\nAdditional links:\n\n* [How to interpret data and prediction drift together?](https://evidentlyai.com/blog/data-and-prediction-drift)\n\n* [Do I need to monitor data drift